<a href="https://colab.research.google.com/github/rawanmt/GradProject/blob/main/pcappeek_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *** How to Start ***


*   Install the Required Packages -> ***BLOCK 1***
*   Excute script.shell -> ***BLOCK 2***
*   Upload the pcap file(s) you want to test 
*   Run the tool script -> ***BlOCK 3***
*   Pass the argument required and the pcap file path then run -> ***BLOCK 4***

The tool will create *'Yourdump.csv'* file which is used to generate the graph, and *'figure.png'* file which is a result of excuting the visualization function












# **BLOCK 1** ↓

In [3]:
%%shell
# Required Packages to run the tool 
pip install scapy
apt-get install tshark
pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tshark is already the newest version (2.6.10-1~ubuntu18.04.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **BLOCK 2** ↓


In [4]:
%%writefile script.sh
# Fields Extraction shell script for visualization 
echo "[o] Making csv.dump ..."; 
tshark -r $1 -Tfields -E separator=, -E header=y -E quote=d -e frame.number -e frame.time -e frame.len -e eth.dst -e eth.src -e ip.version -e ip.proto -e ip.src -e ip.dst -e tcp.srcport -e tcp.dstport -e tcp.flags.res -e tcp.flags.urg  -e tcp.flags.syn -e tcp.flags.ack > Yourdump.csv;
echo "[*] Check your .csv dump in Yourdump.csv";


Writing script.sh


# **BLOCK 3** ↓

In [5]:
%%writefile pcappeek.py 
# the libraries needed
import os
import sys
import subprocess
import time

import collections
import datetime

import argparse
from argparse import RawTextHelpFormatter

from scapy.all import *
import scapy.all as scapy
from scapy.utils import rdpcap
from scapy.all import rdpcap, IP, TCP, ICMP

from collections import Counter
from prettytable import PrettyTable

import pandas as pd
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder

from kaleido.scopes.plotly import PlotlyScope


# ------------------------
# Tool Interface
# ------------------------
def banner():
    # tool name and main interface
    print("""
        Welcom to PcapPeek, Let's Analyze your pcap File!
=======================================================================
                  ____                   ____            __  
                 / __ \_________ _____  / __ \___  ___  / /__
                / /_/ / ___/ __ `/ __ \/ /_/ / _ \/ _ \/ //_/
               / ____/ /__/ /_/ / /_/ / ____/  __/  __/ ,<   
              /_/    \___/\__,_/ .___/_/    \___/\___/_/|_|  
                              /_/ 
                Automated Attack Detection and Visualisation Tool   
                             version: 1.0
=======================================================================
    """)

# ------------------------
# User input parser
# ------------------------
def add_parser_args():
    # using argparser to facilitate choosing the argument for the user
    parser = argparse.ArgumentParser(prog="pcappeek.py", usage='%(prog)s file_name [options]',
                                     epilog="Example: ./%(prog)s -f ./pcap_samples/sample1.pcap --analyze --report ",
                                     formatter_class=RawTextHelpFormatter)
    parser.add_argument("file", type=str, help="Specify the File to be Analyzed")
    parser.add_argument('-s', '--statistics',action="store_true", help="Provides File Statistics; top IP addresses and protocols ")
    parser.add_argument('-a', '--analyze', action="store_true", help="Auto Detection of Attacks in the pcap File")
    parser.add_argument('-v', '--visualize', action="store_true", help="Visualizes Network Traffic")
    return parser

# ------------------------
# opening the pcap file 
# ------------------------
def open_pcap(pcap_file):
  pcap_file = rdpcap(args.file)
  return pcap_file

# ---------------------------------------------
# Functions Definitions show statistics 
# ---------------------------------------------
def show_statistics(pcap_file):
  print("\n[*] Protocols %(pcap_file)s" % {"pcap_file": open_pcap(pcap_file)})
  # read and append ips into a list 
  pkts = rdpcap(pcap_file)
  srcIP = []
  dstIP =[]
  protocol =[]
  for pkt in pkts:
    if IP in pkt:
      try:
          srcIP.append(pkt[IP].src)
          dstIP.append(pkt[IP].dst)
      except:
        pass
  # count the number of IPs found
  srcIP_counter = Counter()
  dstIP_counter = Counter()

  for ip in srcIP:
      srcIP_counter[ip] += 1
  for ip in dstIP:
      dstIP_counter[ip] += 1     
  # Print the IPs with thier count 
  srcIPs_table = PrettyTable(["IP", "Count"])
  dstIPs_table = PrettyTable(["IP", "Count"])
  if len(srcIP) > 1:
    for ip, count in srcIP_counter.most_common():
        if count > 1:
            srcIPs_table.add_row([ip, count])
    print("\n[*] Source IPs found in '{}' and their count:".format(pcap_file))
    print(srcIPs_table)
  else:
    print("\n[*] No Source IPs found in '{}'".format(pcap_file))

  if len(dstIP) > 1:
    for ip, count in dstIP_counter.most_common():
        if count > 1:
            dstIPs_table.add_row([ip, count])
    print("[*] Destination IPs found in '{}' and their count:".format(pcap_file))
    print(dstIPs_table)
  else:
    print("[*] No Destination IPs found in '{}'".format(pcap_file))

# #----------------------------------------------
# # Functions Definitions
# # ---------------------------------------------

# ---------------------------------------------
# Functions Definitions SYN Flooding Detection 
# ---------------------------------------------
count=Counter()
syn_flag = False
def detect_SYN_flooding(pkt):
    global syn_flag
    if TCP in pkt and pkt[TCP].flags == 'S' :
        SYN_flood=pkt.sprintf('{IP:%IP.src%}')
        count[SYN_flood] += 1
        if count.most_common(1)[0][1] > syn_threshold :
            syn_flag = True
            count[SYN_flood] = 0
    if syn_flag :
        print('[*] Suspected SYN Flood from: ' + str(pkt[IP].src) + ' on PORT: ' + str(pkt[TCP].dport))
        syn_flag = False

# -----------------------------------------------
#  Functions Definitions ICMP Flooding Detection 
# -----------------------------------------------
count=Counter()
icmp_flag = False
def detect_ICMP_flooding(pkt):
    global icmp_flag
    if ICMP in pkt and pkt[ICMP].type == 8:
        icmp_flood=pkt.sprintf('{IP:%IP.src%}')
        count[icmp_flood] += 1
        if count.most_common(1)[0][1] > icmp_threshold :
            icmp_flag = True
            count[icmp_flood] = 0
    if icmp_flag :
        print('[*] Suspected ICMP Flood from: ' + str(pkt[IP].src) )
        icmp_flag = False


# -----------------------------------------------
#  Functions Definitions Ping of Death Detection 
# -----------------------------------------------
count=Counter()
icmp_flag = False
icmp_threshold=0

def detect_ICMP_PingofDeath(pkt):
    global icmp_flag
    if ICMP in pkt and pkt[ICMP].type == 8 and len(pkt[ICMP]) > 64:
        icmp_pod=pkt.sprintf('{IP:%IP.src%}')
        count[icmp_pod] += 1
        if count.most_common(1)[0][1] > icmp_threshold :
            icmp_flag = True
            count[icmp_pod] = 0
    if icmp_flag :
        print('[*] Suspected ICMP Ping of Death from: ' + str(pkt[IP].src))
        icmp_flag = False

# -----------------------------------------------
#  Functions Definitions Visualize the traffic 
# -----------------------------------------------
def run_script(pcap_file):
  subprocess.run(['bash','/content/script.sh', pcap_file])

def visualize_traffic(csv_file):
  try: 
    try: 
      df = pd.read_csv(csv_file, skip_blank_lines=True, low_memory=False)
      df.dropna(how="all", inplace=True)

      # Converting Categorical Fields into DataFeames
      # Length
      group_vars = df['frame.len'].unique()
      df_len = pd.DataFrame({'frame.len':df['frame.len'].unique()})
      df_len['length'] = df_len.index
      df = pd.merge(df, df_len, on = 'frame.len', how='left') 

      # Destination MAC
      group_vars = df['eth.dst'].unique()
      df_Dmac = pd.DataFrame({'eth.dst':df['eth.dst'].unique()})
      df_Dmac['Dmac'] = df_Dmac.index
      df = pd.merge(df, df_Dmac, on = 'eth.dst', how='left')

      # Source MAC
      group_vars = df['eth.src'].unique()
      df_Smac = pd.DataFrame({'eth.src':df['eth.src'].unique()})
      df_Smac['Smac'] = df_Smac.index
      df = pd.merge(df, df_Smac, on = 'eth.src', how='left')

      # IP Protocol
      group_vars = df['ip.proto'].unique()
      df_proto = pd.DataFrame({'ip.proto':df['ip.proto'].unique()})
      df_proto['proto'] = df_proto.index
      df = pd.merge(df, df_proto, on = 'ip.proto', how='left')

      # Source IP
      group_vars = df['ip.src'].unique()
      df_Sip = pd.DataFrame({'ip.src':df['ip.src'].unique()})
      df_Sip['Sip'] = df_Sip.index
      df = pd.merge(df, df_Sip, on = 'ip.src', how='left')

      # Dest IP
      group_vars = df['ip.dst'].unique()
      df_Dip = pd.DataFrame({'ip.dst':df['ip.dst'].unique()})
      df_Dip['Dip'] = df_Dip.index
      df = pd.merge(df, df_Dip, on = 'ip.dst', how='left')

      dimensions = list([
                  
                        #  Frame Number
                        dict(range=[df['frame.number'].min(),df['frame.number'].max()],
                              label='Frame Number', values=df['frame.number']),
                        
                        #  Frame length
                        dict(range=[df['frame.len'].min(),df['frame.len'].max()],
                              label='Frame Length', values=df['frame.len']),

                        #  Source MAC
                        dict(range=[0,df['Smac'].max()],
                            tickvals = df_Smac['Smac'], ticktext = df_Smac['eth.src'],
                            label='Source MAC', values=df['Smac']),

                        #  Destination MAC
                        dict(range=[0,df['Dmac'].max()],
                            tickvals = df_Dmac['Dmac'], ticktext = df_Dmac['eth.dst'],
                            label='Destination MAC', values=df['Dmac']),
                        
                        #  Source IP
                        dict(range=[0,df['Sip'].max()],
                            tickvals = df_Sip['Sip'], ticktext = df_Sip['ip.src'],
                            label='Source IP', values=df['Sip']),

                        #  Destination IP
                        dict(range=[0,df['Dip'].max()],
                            tickvals = df_Dip['Dip'], ticktext = df_Dip['ip.dst'],
                            label='Destination IP', values=df['Dip']),
                         
                        #  IP Protocol
                        dict(range=[0,df['proto'].max()],
                            tickvals = df_proto['proto'], ticktext = df_proto['proto'],
                            label='IP Protocol', values=df['proto']),
                         
                        #  #  Proto
                        # dict(range=[df['ip.proto'].min(),df['ip.proto'].max()],
                        #       label='IP Protocol', values=df['ip.proto']),
                         
                        #  Source Port
                        dict(range=[df['tcp.srcport'].min(),df['tcp.srcport'].max()],
                              label='Source Port', values=df['tcp.srcport']), 

                          #  Destination Port
                        dict(range=[df['tcp.dstport'].min(),df['tcp.dstport'].max()],
                              label='Destination Port', values=df['tcp.dstport']), 

                        #  tcp.flags.syn
                        dict(range=[df['tcp.flags.syn'].min(), df['tcp.flags.syn'].max()],
                              label='tcp.flags.syn', values=df['tcp.flags.syn']),

                        #  tcp.flags.ack
                        dict(range=[df['tcp.flags.ack'].min(), df['tcp.flags.ack'].max()],
                              label='tcp.flags.ack', values=df['tcp.flags.ack']),                 
                      
                        ])

      fig = go.Figure(data=go.Parcoords(line = dict(color = df['frame.number'],
                        colorscale = 'ice'), 
                        dimensions=dimensions))

      fig.update_layout(width=1200, height=1000,margin=dict(l=150, r=60, t=60, b=40))
  
      try:
        scope = PlotlyScope(
            plotlyjs="https://cdn.plot.ly/plotly-latest.min.js",
        )

        with open("figure.png", "wb") as f:
          f.write(scope.transform(fig, format="png"))

        print("[*] Graph File has been Generated, Visualization Completed")

      except Exception as FileException:
        print("[X] An Error Occurred, Cannot Generate the Graph File")

    except pd.errors.EmptyDataError:
      print("[X] CSV File is Empty, Cannot Visualize the Traffic")

  except FileNotFoundError:
    print("[X] Error Creating CSV File, Cannot Visualize the Traffic")

# ------------------------
#    Main Starts Here
# ------------------------
if __name__ == '__main__':
    banner()
    parser = add_parser_args()
    args= parser.parse_args()
    # if the file is supplied as an argument:
    if args.file:
      try:
          # open the pcap and print its name and the number of packets in the file it will exit if the file isn't supported
          print("[*] Opening %(filename)s... \n" % {"filename": args.file})
            # -------------------
          if args.statistics:
            try:
              show_statistics(args.file)
            except:
                print("[*] Error Getting file statistics")
              
          # -------------------
          # opening the pcap file for detection 
          pkts = open_pcap(args.file)
        
          # -------------------
          if args.analyze:

            start_time = time.time()

            # arp spoofing:
            print("[*] SCANNING for ARP Spoofing..")
            #global variabled
            mac = ""
            ip = ""
            macIpMap = {}
            # defining function to process packet
            def process_sniffed_packet(packet):
              # Format current date & time into string 
                global mac, ip, macIpMap
                # get mac and ip address of packet
                mac = packet.hwsrc
                ip = packet.psrc
                # comparing values of MAC Addresses with map macIpMap[ip]
                if any(ip in map for map in macIpMap):
                    if ARP in packet and packet[ARP].op in (1,2):
                        if macIpMap[ip] != mac :
                            attacked = True
                        else:
                            attacked = False
                else:
                    macIpMap[ip] = mac
                    attacked = False
                return attacked
            for pkt in pkts:
              try:
                  attacked = process_sniffed_packet(pkt)
                  if attacked:
                      dateTime = (datetime.fromtimestamp(pkt.time)).strftime("%a, %d %b %Y, %H:%M:%S")
                      print("[*] Suspected ARP Spoofing Attack Detected !")
                      print(f"[-] {dateTime}\n[-] Real IP: {ip}\n[-] Real MAC: {macIpMap[ip]}\n[-] Fake MAC (Attacker): {mac}")
                      print("[*] SCANNING for ARP Spoofing...\n")
              except:
                pass

            print("[*] ARP Spoofing Detection Completed\n")

            # SYN Flooding:
            try:
              print("[*] SCANNING for SYN Flooding...\n")
                  # Ask the user to specify the threshold of SYN packets
              syn_threshold=int(input("[..] Please Choose a SYN Pakets Threshold: "))
                  # throw an error if the value entered is not an int and ask the user to enter again

              for pkt in pkts:
                detect_SYN_flooding(pkt) # SYN Flood Detection

            except ValueError:
              print("[X] Enter a Valid Value")
              syn_threshold=int(input("[..] Please Choose a SYN Pakets Threshold: "))
              print("[*] SCANNING for SYN Flooding...")
      
            print("[*] SYN Flood Detection Completed\n")

            
            try:
              # ICMP flooding
              
              print("[*] SCANNING for ICMP Flooding...")
                # Ask the user to specify the threshold of SYN packets
              icmp_threshold=int(input("[..] Please Choose an ICMP Pakets Threshold: "))
                # throw an error if the value entered is not an int and ask the user to enter again
              for pkt in pkts:
                detect_ICMP_flooding(pkt) # ICMP Flood Detection

            except ValueError:
              print("[X] Enter a Valid Value")
              syn_threshold=int(input("[..] Please Choose an ICMP Pakets Threshold: "))
              print("[*] SCANNING for ICMP Flooding...")

            print("[*] ICMP Flood Detection Completed\n")
        
            try:
              # Ping of death:

              print("[*] SCANNING for Ping of Death...")
              for pkt in pkts:
                detect_ICMP_PingofDeath(pkt) # ICMP Flood Detection

            except:
                  print("[*] Error detecting ping of death ")

            print("[*] Ping of Death Detection Completed\n")
            
            print("[*] Total Execuiton Time: ", (time.time() - start_time))

          # ----------------
          if args.visualize:
            start_time = time.time()
            print("[*] Visualizing Network Traffic of {} ...".format(args.file))
            run_script(args.file)
            visualize_traffic('/content/Yourdump.csv')
            print("[*] Total Execuiton Time: ", (time.time() - start_time))

        # if there is no file chosen when running the tool

      except Exception as FileException:
                    # prints when the file doesn't exist or is of an invalid typ
              print('"{}" does not exist'.format(args.file), file=sys.stderr)
              print("[X] Please Choose a Valid pcap File")



      except Exception as FileException:
          # prints when the file doesn't exist or is of an invalid typ
          print('"{}" does not exist'.format(args.file), file=sys.stderr)
          print("[X] Please Choose a Valid pcap File")
         

Overwriting pcappeek.py


# **BLOCK 4** ↓

In [6]:
# run the program with the arguments 
!python3 pcappeek.py -h


        Welcom to PcapPeek, Let's Analyze your pcap File!
                  ____                   ____            __  
                 / __ \_________ _____  / __ \___  ___  / /__
                / /_/ / ___/ __ `/ __ \/ /_/ / _ \/ _ \/ //_/
               / ____/ /__/ /_/ / /_/ / ____/  __/  __/ ,<   
              /_/    \___/\__,_/ .___/_/    \___/\___/_/|_|  
                              /_/ 
                Automated Attack Detection and Visualisation Tool   
                             version: 1.0
    
usage: pcappeek.py file_name [options]

positional arguments:
  file              Specify the File to be Analyzed

optional arguments:
  -h, --help        show this help message and exit
  -s, --statistics  Provides File Statistics; top IP addresses and protocols 
  -a, --analyze     Auto Detection of Attacks in the pcap File
  -v, --visualize   Visualizes Network Traffic

Example: ./pcappeek.py -f ./pcap_samples/sample1.pcap --analyze --report 
